In [1]:
from whoisthat.whoisthat import get_summary, spoiler_check

In [2]:
# import yaml
# with open('whoisthat/database.yml', 'r') as file:
#     print(yaml.safe_load(file))

# Generate the summary
- Place a bookmark in your book
- Generate a summary of the character of interest

In [3]:
book = 'Pride and Prejudice'
bookmark = 'Volume I' # e.g. Volume or Chapter
character = 'Mr. Wickham'
pride_and_prejudice_summary = get_summary(book, bookmark, character)
print(pride_and_prejudice_summary)

Here is a summary of Mr. Wickham's character based on Volume I:

Arrived at Netherfield Park as a charming and flirtatious officer who has a reputation for being reckless and disreputable, having previously eloped with Georgiana Darcy.


In [4]:
pride_and_prejudice_spolier_check = spoiler_check(book, character, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

Yes. The summary does not mention Lydia Bennet, but it mentions Georgiana Darcy, indicating that Mr. Wickham has already eloped with someone (Georgiana) before arriving at Netherfield Park, which is later revealed to be Lydia Bennet in the rest of the novel.


# A Game of Thrones

In [5]:
book = 'A Game of Thrones'
bookmark = 'Chapter 1'
character = 'Jon Snow'
game_of_thrones_summary = get_summary(book, bookmark, character)
print(game_of_thrones_summary)

Here is a summary of Jon Snow's character up to Chapter 1:

Jon Snow is a bastard son of Ned Stark, who has been raised at Winterfell and trained in combat by Master-at-Arms Jeor Mormont.


In [6]:
print(spoiler_check(book, character, game_of_thrones_summary))

No.

This is because the summary only mentions Jon Snow's parentage as being raised by Ned Stark, without mentioning Lyanna Stark and Rhaegar Targaryen as his parents. The information about Lyanna and Rhaegar is a major plot twist revealed later in the series.


In [7]:
tyrion_summary = get_summary(book, bookmark, 'Tyrion Lannister')
print(spoiler_check(book, 'Tyrion Lannister', tyrion_summary))

No spoilers in the database for Tyrion Lannister in A Game of Thrones.


# Harry Potter

In [8]:
book = 'Harry Potter and the Half-Blood Prince'
character = 'Albus Dumbledore'

chapter_20_summary = get_summary(book, 'Chapter 20', character)
print("Chapter 20 summary:")
print(chapter_20_summary)
print("")
print("")
print("Spoiler check for Chapter 20:")
print(spoiler_check(book, character, chapter_20_summary))
print("")
print("")

chapter_29_summary = get_summary(book, 'Chapter 29', character)
print("Chapter 29 summary:")
print(chapter_29_summary)
print("")
print("")
print("Spoiler check for Chapter 29:")
print(spoiler_check(book, character, chapter_29_summary))

Chapter 20 summary:
Here is a summary of Dumbledore's character in 20 words or less:

Dumbledore: Wise Headmaster who protected Harry, revealed truth to him, and made sacrifices for greater good, while keeping some secrets.


Spoiler check for Chapter 20:
Yes.

The reason is because the summary mentions that Dumbledore "made sacrifices for greater good", which implies his death, and the presence of this information would indeed be a spoiler.


Chapter 29 summary:
Here is a summary of Albus Dumbledore's character in 20 words or less:

Dumbledore, the Hogwarts headmaster, has protected Harry Potter, demonstrated exceptional wisdom and power, and made difficult sacrifices for the greater good.


Spoiler check for Chapter 29:
Yes.

The summary mentions that Dumbledore "made difficult sacrifices for the greater good", which hints at his eventual death, which was indeed caused by Severus Snape in the book.
